# Back Propagation Implementation

In [1]:
import numpy as np
import scipy.optimize as op
import scipy.io

In [2]:
mat = scipy.io.loadmat('ex4data1.mat')

In [3]:
# X is a 5000 x 400 matrix. Each row is an "unrolled" 20 x 20 pixel image of a digit
# using numbers to expresses the image's greyscale value at that pixel. 

X = mat['X']

# y is a 5000 x 1 vector containing the correct label for each example in the training set.
# the number 0 is given the label 10 in this data.

y = mat['y']

y = y.ravel()

In [4]:
def sigmoid(z):
    
    # Activation function.
    
    return 1/(1+np.exp(np.negative(z)))  

In [5]:
def sigmoid_gradient(z):
    
    # Derivative of activation function.
    
    return sigmoid(z) * (1-sigmoid(z))

In [6]:
def add_bias_feature(X):
    
    # Adds bias feature of 1's to X
    
    m,n = np.shape(X)
    X_0 = np.ones(len(X)).reshape(m,1)
    
    return np.append(X_0, X, 1)

In [7]:
def pack_theta(t1, t2):
    
    # Packs two theta matricies to one long theta vector.
    
    return np.concatenate((t1.ravel(), t2.ravel()))

def unpack_theta(theta_vec, input_layer_size, hidden_layer_size, num_labels):
    
    # Rearranges theta vector into 2 matrices.
    
    t1_start = 0
    t1_end = hidden_layer_size * (input_layer_size + 1)
    t1 = theta_vec[t1_start:t1_end].reshape((hidden_layer_size, input_layer_size + 1))
    t2 = theta_vec[t1_end:].reshape((num_labels, hidden_layer_size + 1))
    
    return t1, t2

In [8]:
def initialize_theta(L_out, L_in):
    
    # Create initial weight matrix with values between +/- epsilon
    
    epsilon = 0.12
    
    return np.random.rand(L_out, L_in + 1) * (2*epsilon) - epsilon

In [9]:
def cost_function(theta_vec, X, y, input_layer_size, hidden_layer_size, num_labels, lambda_):
    
    # NN cost function. 
    # Returns cost and gradient.
    
    # Reformat theta vector into weight matrices
    theta1, theta2 = unpack_theta(theta_vec, input_layer_size, hidden_layer_size, num_labels)
    
    m = len(X)
    
    # Forward propagation
    a1 = add_bias_feature(X)
    z2 = theta1.dot(a1.T)
    a2 = sigmoid(z2.T)
    a2 = add_bias_feature(a2)
    z3 = theta2.dot(a2.T)
    a3 = sigmoid(z3.T)
    
    cost = sum(sum((1/m) * (-y * np.log(a3) - (1-y) * np.log(1-a3)))) + \
            + (lambda_/(2*m)) * (sum(sum(theta1[:,1:]**2)) + sum(sum(theta2[:,1:]**2)))
        
    theta2_grad = np.zeros(theta2.shape)
    theta1_grad = np.zeros(theta1.shape)
    
    delta_1 = np.zeros(theta1.shape)
    delta_2 = np.zeros(theta2.shape)
    
    # Back Propagation
    for i in range(m):
        # Error for activation layer 3
        d3 = a3[i, :] - y[i, :]
        # Error for activation layer 2
        d2 = theta2[:,1:].T.dot(d3) * sigmoid_gradient(z2[:,i])
        # Accumulate gradient
        delta_2 += np.outer(d3, a2[i,:])
        delta_1 += np.outer(d2, a1[i,:])
    
    theta1_grad[:, 0] = delta_1[:, 0] / m
    theta1_grad[:, 1:] = delta_1[:, 1:] / m + (theta1[:, 1:] * (lambda_/m))
    theta2_grad[:, 0] = delta_2[:, 0] / m
    theta2_grad[:, 1:] = delta_2[:, 1:] / m + (theta2[:, 1:] * (lambda_/m))

    grad = pack_theta(theta1_grad, theta2_grad)
    
    return cost, grad

In [10]:
def compute_numerical_gradient(J, theta):
    
    # Uses 2 sided difference to approximate the derivative of the cost function
    # in order to evaluate the implementation of back propagation.
    
    epsilon = 10e-4
    num_grad = []
    perturb = np.zeros(len(theta))
    
    for i in range(len(theta)):
        perturb[i] = epsilon
        theta_plus = theta
        theta_minus = theta
        theta_plus = theta_plus + perturb
        theta_minus = theta_minus - perturb
        
        loss1, _ = J(theta_minus)
        loss2, _ = J(theta_plus)
        
        num_grad.append((loss2 - loss1) / (2*epsilon))
        
        perturb[i] = 0
        
    return np.array([num_grad])

In [11]:
def check_nn_gradient(lambda_):

    # Creates a small neural network in order to compare the implementation 
    # of back propagation with the numerical gradient approximation.
    
    input_layer_size = 3
    hidden_layer_size = 5
    num_labels = 3
    m = 5
    
    X = initialize_theta(m, input_layer_size -1)
    y = np.random.randint(0,3, m)
    y_nn = np.zeros([m, num_labels])
    for i in range(len(y)):
        y_nn[i, y[i] - 1] = 1
    
    theta1 = initialize_theta(hidden_layer_size, input_layer_size)
    theta2 = initialize_theta(num_labels, hidden_layer_size)
    theta_vec = pack_theta(theta1, theta2)

    cost, grad = cost_function(theta_vec, 
                               X, 
                               y_nn, 
                               input_layer_size, 
                               hidden_layer_size, 
                               num_labels, 
                               lambda_)

    num_grad = compute_numerical_gradient(lambda theta: cost_function(theta, 
                                                        X, 
                                                        y_nn, 
                                                        input_layer_size, 
                                                        hidden_layer_size, 
                                                        num_labels, 
                                                        lambda_), theta_vec)
    
    diff = np.linalg.norm(num_grad - grad)/np.linalg.norm(num_grad + grad)
    
    print('Difference between numerical and computed gradients: {}'.format(diff))

In [12]:
# Check to make sure numerical gradients match up to those computed in the cost function
# (difference should be very small).

check_nn_gradient(1)

Difference between numerical and computed gradients: 2.019370913255408e-09


In [13]:
m = len(X)
input_layer = 400
hidden_layer = 25
num_labels = 10
lambda_ = 3

In [14]:
# Convert y to a 5000 X 10 matrix, where each row indicates membership to a digit class.

y_nn = np.zeros([5000, 10])
    
for i in range(len(y)):
    y_nn[i, y[i] - 1] = 1

In [15]:
# Initialize weights.

initial_theta1 = initialize_theta(L_in = 400, L_out=25)
initial_theta2 = initialize_theta(L_in = 25, L_out = 10)

initial_theta_vec = pack_theta(initial_theta1, initial_theta2)

In [16]:
# Minimize cost function.

result = op.minimize(fun=cost_function, 
                             x0=initial_theta_vec, 
                             args=(X, y_nn, input_layer, hidden_layer, num_labels, 3), 
                             method= 'TNC', 
                             jac=True, options = {'maxiter' : 400, 'disp': True})

In [17]:
optimal_theta = result.x

In [18]:
optimal_t1, optimal_t2 = unpack_theta(optimal_theta, input_layer, hidden_layer, num_labels)

In [19]:
def predict_NN(t1, t2, X, y):
    
    # Performs forward propagation and returns accuracy of predictions.
    
    a1 = add_bias_feature(X)
    z2 = t1.dot(a1.T)
    a2 = sigmoid(z2.T)
    a2 = add_bias_feature(a2)
    z3 = t2.dot(a2.T)
    a3 = sigmoid(z3.T)
    
    predictions = np.argmax(a3, axis = 1) + 1
    accuracy =  np.mean((predictions == y) * 1)
    
    return 'Training accuracy for neural network: {:0.2f}%'.format(accuracy * 100)

In [20]:
predict_NN(optimal_t1, optimal_t2, X, y)

'Training accuracy for neural network: 97.60%'